In [27]:
import pandas as pd
import numpy as np

In [41]:
classesDF  = pd.read_csv(r"C:\Users\User\Desktop\UAB\3rd-year\2nd-semester\synthesis project II\elliptic_bitcoin_dataset\elliptic_txs_classes.csv")

# Remove all nodes that have unknown class
classesDF = classesDF[classesDF['class'] != "unknown"]

# If class = 2, then map to 0, else map to 1
classesDF['class'] = classesDF['class'].map({'2': 0, '1': 1})

classesDF.head(5)

,txId,class
3,232438397,0
9,232029206,0
10,232344069,0
11,27553029,0
16,3881097,0


In [42]:
edgesDF = pd.read_csv(r"C:\Users\User\Desktop\UAB\3rd-year\2nd-semester\synthesis project II\elliptic_bitcoin_dataset\elliptic_txs_edgelist.csv")

edgesDF.head(5)

,txId1,txId2
0,230425980,5530458
1,232022460,232438397
2,230460314,230459870
3,230333930,230595899
4,232013274,232029206


In [43]:
featuresDF = pd.read_csv(r"C:\Users\User\Desktop\UAB\3rd-year\2nd-semester\synthesis project II\elliptic_bitcoin_dataset\elliptic_txs_features.csv", header=None)

# Change column names --> Column 1 is txId, Column 2 is timestep and the rest are unknown features
featuresDF.columns = ['txId', 'timestep'] + ['f' + str(i) for i in range(165)]

# Remove all edges that do not appear in classesDF
featuresDF = featuresDF[featuresDF['txId'].isin(classesDF['txId'])]

featuresDF.head(5)

,txId,timestep,f0,f1,f2,f3,f4,f5,f6,f7,...,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164
3,232438397,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,...,-0.577099,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792
9,232029206,1,-0.005027,0.578941,-0.091383,4.380281,-0.063725,4.667146,0.851305,-0.163645,...,-0.577099,-0.613614,0.241128,0.241406,0.604120,0.008632,-0.131155,0.333211,-0.120613,-0.119792
10,232344069,1,-0.147852,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.137933,...,-0.577099,-0.613614,0.241128,0.241406,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
11,27553029,1,-0.151357,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.141519,...,-0.539735,-0.582077,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
16,3881097,1,-0.172306,-0.184668,-1.201369,0.028105,-0.043875,-0.029140,0.242712,-0.163640,...,-0.577099,-0.600999,0.241128,0.241406,0.018279,-0.068266,-0.084674,-0.054450,-1.760926,-1.760984


In [44]:
# Append the class to the featuresDF based on txID
featuresDF = featuresDF.merge(classesDF, on='txId')

# Move features 'class' to first column
cols = list(featuresDF.columns)
cols = cols[:1] + [cols[-1]] + cols[1:-1]
featuresDF = featuresDF[cols]

featuresDF.head(5)

,txId,class,timestep,f0,f1,f2,f3,f4,f5,f6,...,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164
0,232438397,0,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,...,-0.577099,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792
1,232029206,0,1,-0.005027,0.578941,-0.091383,4.380281,-0.063725,4.667146,0.851305,...,-0.577099,-0.613614,0.241128,0.241406,0.604120,0.008632,-0.131155,0.333211,-0.120613,-0.119792
2,232344069,0,1,-0.147852,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,-0.577099,-0.613614,0.241128,0.241406,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
3,27553029,0,1,-0.151357,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,-0.539735,-0.582077,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
4,3881097,0,1,-0.172306,-0.184668,-1.201369,0.028105,-0.043875,-0.029140,0.242712,...,-0.577099,-0.600999,0.241128,0.241406,0.018279,-0.068266,-0.084674,-0.054450,-1.760926,-1.760984


In [45]:
"""
connections_dict = {}



for _, row in edgesDF.iterrows():
    txId1, txId2 = row['txId1'], row['txId2']
    
    if txId1 not in connections_dict:
        connections_dict[txId1] = []
    if txId2 not in connections_dict:
        connections_dict[txId2] = []
        
    connections_dict[txId1].append(txId2)
    connections_dict[txId2].append(txId1)

# Función para obtener la lista de conexiones de un txId
def get_connections(txId):
    return connections_dict.get(txId, [])

# Aplicar la función a cada fila de featuresDF para crear la nueva columna 'edges'
featuresDF['edges'] = featuresDF['txId'].apply(get_connections)

featuresDF.head()"""

"\nconnections_dict = {}\n\n\n\nfor _, row in edgesDF.iterrows():\n    txId1, txId2 = row['txId1'], row['txId2']\n    \n    if txId1 not in connections_dict:\n        connections_dict[txId1] = []\n    if txId2 not in connections_dict:\n        connections_dict[txId2] = []\n        \n    connections_dict[txId1].append(txId2)\n    connections_dict[txId2].append(txId1)\n\n# Función para obtener la lista de conexiones de un txId\ndef get_connections(txId):\n    return connections_dict.get(txId, [])\n\n# Aplicar la función a cada fila de featuresDF para crear la nueva columna 'edges'\nfeaturesDF['edges'] = featuresDF['txId'].apply(get_connections)\n\nfeaturesDF.head()"

In [46]:
in_degree_dict = {}
out_degree_dict = {}

for _, row in edgesDF.iterrows():
    txId1, txId2 = row['txId1'], row['txId2']
    
    if txId2 not in in_degree_dict:
        in_degree_dict[txId2] = []
    in_degree_dict[txId2].append(txId1)
    
    if txId1 not in out_degree_dict:
        out_degree_dict[txId1] = []
    out_degree_dict[txId1].append(txId2)

# Filtrar los diccionarios para incluir solo IDs presentes en featuresDF_example
valid_ids = set(featuresDF['txId'])

in_degree_dict_filtered = {k: [v for v in vals if v in valid_ids] for k, vals in in_degree_dict.items() if k in valid_ids}
out_degree_dict_filtered = {k: [v for v in vals if v in valid_ids] for k, vals in out_degree_dict.items() if k in valid_ids}

# Funciones para obtener las listas de conexiones
def get_in_degree_connections(txId):
    return in_degree_dict_filtered.get(txId, [])

def get_out_degree_connections(txId):
    return out_degree_dict_filtered.get(txId, [])

# Aplicar las funciones para crear las nuevas columnas
featuresDF['in_degree_edges'] = featuresDF['txId'].apply(get_in_degree_connections)
featuresDF['out_degree_edges'] = featuresDF['txId'].apply(get_out_degree_connections)

featuresDF.head() 

,txId,class,timestep,f0,f1,f2,f3,f4,f5,f6,...,f157,f158,f159,f160,f161,f162,f163,f164,in_degree_edges,out_degree_edges
0,232438397,0,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,...,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792,"[232047899, 3877118, 230456717, 230456719, 387...",[92491280]
1,232029206,0,1,-0.005027,0.578941,-0.091383,4.380281,-0.063725,4.667146,0.851305,...,0.241128,0.241406,0.604120,0.008632,-0.131155,0.333211,-0.120613,-0.119792,"[230391412, 91411167, 232346257, 230411642, 23...",[5119722]
2,232344069,0,1,-0.147852,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,0.241128,0.241406,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,[],"[27553029, 234419685]"
3,27553029,0,1,-0.151357,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,[232344069],[230411688]
4,3881097,0,1,-0.172306,-0.184668,-1.201369,0.028105,-0.043875,-0.029140,0.242712,...,0.241128,0.241406,0.018279,-0.068266,-0.084674,-0.054450,-1.760926,-1.760984,[89273],[232457116]


In [47]:
ex = featuresDF[featuresDF['txId'] == 232438397] 

In [48]:
ex

,txId,class,timestep,f0,f1,f2,f3,f4,f5,f6,...,f157,f158,f159,f160,f161,f162,f163,f164,in_degree_edges,out_degree_edges
0,232438397,0,1,0.163054,1.96379,-0.646376,12.409294,-0.063725,9.782742,12.414558,...,0.241128,0.241406,1.072793,0.08553,-0.131155,0.677799,-0.120613,-0.119792,"[232047899, 3877118, 230456717, 230456719, 387...",[92491280]
